# Výpočet kolokací vlastní funkcí podle Diceova koeficientu

In [ ]:
import nltk
from nltk.book import *

## Motivační příklad

Slova "holy" a "am" se v textu *Monty Python and the Holy Grail* (`text6`) obě objevují asi 20×, přičemž "holy" je skoro vždy v kombinaci s (bezprostředně následujícím) "grail", a "am" je skoro vždy v kombinaci s (bezprostředně předcházejícím) "I".

In [ ]:
text6.concordance("holy")

In [ ]:
text6.concordance("am")

Který z těchto **bigramů** (dvojic sousedících slov) intuitivně tvoří silnější **kolokaci**?

Z pohledu gramatiky by se mohlo zdát, že je to "I am" -- cítíme, že jde o spojení slov takřka učebnicové (skutečně ve smyslu, že na ně narazíme v učebnicích). Ovšem sílu kolokace bychom měli posuzovat spíše z hlediska lexikonu, tj. do jaké míry vytváří sledovaná jednotka ucelené a ustálené víceslovné **pojmenování** či **frázi**. Pro tento pohled jsou důležitá dvě kritéria -- zkoumané spojení slov by mělo být:

1. **frekventované** (o jisté míře ustálenosti, tedy lexikalizace, lze uvažovat pouze u jednotek, které se v textech vyskytují opakovaně)
2. **výlučné** (slova v rámci spojení by se měla spolu vyskytovat nápadně častěji než ve společnosti jiných slov)

Když na "I am" a "holy grail" pohlédneme touto optikou, zjistíme, že obě spojení jsou podobně frekventovaná (kolem 20 výskytů), ale spojení "I am" je mnohem méně výlučné, neboť slovo "I" se v textu vyskytuje celkem 260× (tj. ve většině výskytů je v okolí jiných slov než "am"):

In [ ]:
text6.concordance("I")

Naopak "grail" je v textu 39×, z čehož plyne, že souvýskyty se slovem "holy" tvoří mnohem signifikantnější proporci jeho výskytů. "holy grail" je tedy v textu `text6` silnější kolokací než "I am".

In [ ]:
text6.concordance("grail")

## Implementace

Abychom mohli kvantifikovat, *jak přesně* silnější, musíme použít nějakou **asociační** neboli **kolokační míru**. Měrou, která dobře zohledňuje obě výše uvedená kritéria (frekventovanost i výlučnost), je tzv. **Diceův koeficient** (resp. k plnému zohlednění kritéria frekventovanosti je dobré málo frekventované bigramy za kandidáty na kolokace vůbec nepovažovat). Jeho definici můžeme vyčíst z literatury nebo např. [z wiki ÚČNK](http://wiki.korpus.cz/doku.php/pojmy:asociacni_miry?redirect=1#dice_a_logdice), níže je uvedená jedna z možných implementací v Pythonu.

**POZN.:** Není to implementace nejefektivnější (např. kvůli tomu, že spočítáme frekvenční distribuci všech unigramů i bigramů v textu, ačkoli potřebujeme znát jen frekvence dvou cílových slov a jejich bigramu), ale konceptuálně nejlépe odpovídá popisu uvedenému na wiki, takže jako první pokus o převod Diceova koeficientu do Pythonu by měla mít tu výhodu, že je tam relativně malá pravděpodobnost, že člověk někde udělá chybu. Doporučuju porovnat si definici Diceova koeficientu na wiki s touto implementací, abyste se sami přesvědčili, že se jedná skutečně o tentýž výpočet.

In [ ]:
def dice_score(word1, word2, text):
    word1, word2 = word1.lower(), word2.lower()
    bigram_fd = nltk.FreqDist(nltk.bigrams(w.lower() for w in text))
    unigram_fd = nltk.FreqDist(w.lower() for w in text)
    return 2 * bigram_fd[(word1, word2)] / (unigram_fd[word1] + unigram_fd[word2])

# POZN.: funkce nltk.bigrams() si jako argument vezme seznam a vrací postupně všechny
# páry sousedících položek:
list(nltk.bigrams([1, 2, 3, 4, 5]))

Diceův koeficient nabývá hodnot mezi 0 a 1; **čím vyšší** číslo, **tím výlučnější** daná kombinace je. Můžeme si tedy exaktně ověřit náš intuitivní dojem, že "holy grail" je silnější kolokací než "I am":

In [ ]:
dice_score("holy", "grail", text6)

In [ ]:
dice_score("I", "am", text6)

Když máme hotovou prvotní spolehlivou implementaci Diceova koeficientu, můžeme zvážit, zda se nám vyplatí implementovat i hůře srozumitelnou, zato efektivnější verzi. Jaké má původní interpretace nevýhody?

1. je pomalá, protože korpus procházíme zbytečně dvakrát -- poprvé když vytváříme frekvenční distribuci bigramů, podruhé pro frekvenční distribuci unigramů; přitom by mělo být možné získat kýžené informace jediným průchodem korpusu
2. zabírá zbytečně moc paměti, protože vytváříme kompletní frekvenční distribuce všech bigramů a unigramů v korpusu, ačkoli nás zajímají jen dva unigramy a jeden bigram

Na základě těchto poznatků můžeme tedy zkusit implementovat novou, úspornější verzi:

In [ ]:
def dice_score2(word1, word2, text):
    word1, word2 = word1.lower(), word2.lower()
    bigram_f = 0
    word1_f = 0
    word2_f = 0
    for b1, b2 in nltk.bigrams(text):
        b1, b2 = b1.lower(), b2.lower()
        if (word1, word2) == (b1, b2):
            bigram_f += 1
        if b1 == word1:
            word1_f += 1
        elif b1 == word2:
            word2_f += 1
    if b2 == word1:
        word1_f += 1
    elif b2 == word2:
        word2_f += 1
    return 2 * bigram_f / (word1_f + word2_f)

Ta už je složitější a hůře srozumitelná -- nepoužívá intuitivně pojmenované funkce jako `FreqDist`, ale generický for-cyklus, je tedy potřeba do detailu přečíst celý kód, aby člověk získal představu, co se v něm děje. Je tedy také vyšší pravděpodobnost, že v kódu uděláme nějakou chybu (už jen kvůli tomu, že je delší -- každé ťuknutí do klávesnice představuje příležitost pro chybu). Proto je dobře, že máme k dispozici dříve definovanou a "spolehlivější" funkci `dice_score`: můžeme porovnat výsledky obou verzí a ujistit se tak, že jsou stejné a že v té nové jsme neudělali chybu:

In [ ]:
dice_score("holy", "grail", text6), dice_score2("holy", "grail", text6)

In [ ]:
dice_score("I", "am", text6), dice_score2("I", "am", text6)

Výhodou `dice_score2` by mělo být, že zabere méně paměti a bude rychlejší. Rychlost můžeme otestovat pomocí speciální anotace `%timeit`, která následující výraz spustí vícekrát a zobrazí, jak dlouho v průměru trvalo ho vyhodnotit:

In [ ]:
%timeit dice_score("holy", "grail", text6)

In [ ]:
%timeit dice_score2("holy", "grail", text6)

Vidíme, že `dice_score2` je skutečně o něco rychlejší, byť ne až zas tak dramaticky (na takto krátkých textech to výrazně nepocítíme). Jak jsme na tom s pamětí? Využití paměti operacemi v rámci funkce můžeme změřit pomocí anotace `%mprun`, která ale není zabudovaná a tedy automaticky dostupná, musíme ji nejprve nahrát:

In [ ]:
%load_ext memory_profiler

Anotace `%mprun` umožňuje tzv. *profilovat* spotřebu paměti libovolné funkce. Z technických důvodů je ale potřeba, aby tato funkce byla definovaná v samostatném pythonovském modulu, ne v notebooku jako je tento. Naštěstí naše funkce můžeme snadno uložit do dočasného pomocného modulu (= plaintextového souboru, který můžeme pojmenovat např. `tmp.py`) pomocí anotace `%%writefile`:

In [ ]:
%%writefile tmp.py
import nltk

def dice_score(word1, word2, text):
    word1, word2 = word1.lower(), word2.lower()
    bigram_fd = nltk.FreqDist(nltk.bigrams(w.lower() for w in text))
    unigram_fd = nltk.FreqDist(w.lower() for w in text)
    return 2 * bigram_fd[(word1, word2)] / (unigram_fd[word1] + unigram_fd[word2])

def dice_score2(word1, word2, text):
    word1, word2 = word1.lower(), word2.lower()
    bigram_f = 0
    word1_f = 0
    word2_f = 0
    for b1, b2 in nltk.bigrams(text):
        b1, b2 = b1.lower(), b2.lower()
        if (word1, word2) == (b1, b2):
            bigram_f += 1
        if b1 == word1:
            word1_f += 1
        elif b1 == word2:
            word2_f += 1
    if b2 == word1:
        word1_f += 1
    elif b2 == word2:
        word2_f += 1
    return 2 * bigram_f / (word1_f + word2_f)

In [ ]:
from tmp import dice_score, dice_score2

In [ ]:
%mprun -f dice_score dice_score("holy", "grail", text6)

In [ ]:
%mprun -f dice_score2 dice_score2("holy", "grail", text6)

Jak tyhle výpisy číst? Sloupec *Increment* uvádí, jak se na daném řádku zvětšila či zmenšila spotřeba paměti oproti předchozímu řádku. Ovšem pozor, první řádek budeme ignorovat, na něm je v tomto sloupci uvedené počáteční množství zabrané paměti ve chvíli, kdy se funkce začne vykonávat. Jinými slovy, to, že je na prvním řádku číslo v tomto sloupci nejvyšší, **neznamená**, že je to řádek z hlediska paměti nejnáročnější.

Na základě toho vidíme, že jediné řádky, které nějak měřitelně zvyšují spotřebu paměti, jsou ty, kde vytváříme frekvenční distribuce (řádky č. 5 a 6 v `dice_score()`). I tak jde ale vzhledem k celkové spotřebě paměti (sloupec *Mem usage*) znovu o vcelku zanedbatelný rozdíl. To je částečně dané i tím, že jde o hodně krátký text; u 100-milionového či miliardového korpusu bychom rozdíl už patrně pocítili, možná by nám paměť dokonce došla a funkce `dice_score()` by na rozdíl od `dice_score2()` ani nedoběhla.

Poučení tedy zní (slovy známého informatika Donalda Knutha): **Premature optimization is the root of all evil.** Neboli optimalizujte kód (= snažte se upravit implementaci, aby byla účinnější) teprve ve chvíli, kdy vám její pomalost začne vadit jako uživatelům. Neformálně by se tato poučka dala také přeložit "váš čas jakožto programátora je drahocennější než čas počítače" -- takže klidně použijte implementaci, která bude sice pomalejší / paměťožravější, ale bude spolehlivě dobře, než abyste se trápili s vychytáváním chyb (*debugováním*) v implementaci, která bude nepatrně rychlejší / úspornější.

In [ ]:
# pro pořádek smažme dočasný pomocný modul s definicemi funkcí
!rm tmp.py

# Na závěr

Může též nastat situace, kdy předpočítat kompletní frekvenční distribuce unigramů i bigramů bude tím úspornějším řešením -- a to v případě, že nechceme spočítat asociační míru pro konkrétní pár slov, ale vytáhnout z textu všechny bigramy s hodnotou koeficientu Dice vyšší než nějaký práh:

In [ ]:
from operator import itemgetter

# parametr threshold udává práh koeficientu Dice, přes nějž se musí bigram
# přehoupnout, abychom jej považovali za kolokaci (zde jako defaultní hodnota
# celkem arbitrárně zvoleno 0.5); min_freq je požadovaná minimální frekvence
# bigramu, abychom ho vůbec považovali za kandidáta na kolokace (zde znovu
# celkem arbitrárně 3)

def dice_collocations(text, threshold=.5, min_freq=3):
    bigram_fd = nltk.FreqDist((w1.lower(), w2.lower()) for w1, w2 in nltk.bigrams(text))
    unigram_fd = nltk.FreqDist(w.lower() for w in text)
    collocations = []
    for bigram, freq in bigram_fd.items():
        if freq >= min_freq:
            word1, word2 = bigram
            dice = 2 * bigram_fd[bigram] / (unigram_fd[word1] + unigram_fd[word2])
            if dice >= threshold:
                collocations.append((bigram, dice, freq))
    # na závěr ještě kolokace seřadíme sestupně (reverse=True) podle koeficientu
    # Dice a frekvence; protože seřazujeme seznam n-tic podle hodnoty druhého a
    # třetíhu členu n-tice, musíme použít argument key=itemgetter(1, 2), o němž
    # se víc dozvíte v rámci zápočtového úkolu (nebo si taky můžete vyvolat a
    # prostudovat dokumentaci ;) )
    collocations.sort(key=itemgetter(1, 2), reverse=True)
    return collocations

In [ ]:
collocations = dice_collocations(text6)
collocations[:5]

# Post scriptum

Napadá vás ještě, jak funkci pro výpočet koeficientu Dice vylepšit? Mohli bychom např. zkusit uživateli umožnit, aby si mohl vybrat, že kolokáty mohou být i v širším kontextu (ne nutně jen těsně vedle sebe), případně dokonce bez ohledu na pořadí. Jistě vás napadnou další možná rozšíření.